# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:03<00:00,  3.20s/it]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [5]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [8]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [9]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [15]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [16]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [17]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [18]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

2024-05-23 11:10:20,185 - 6363607040 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:10:20,186 - 6380433408 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:10:20,186 - 6363607040 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:10:20,187 - 6380433408 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

2024-05-23 11:10:23,312 - 6380433408 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:10:23,312 - 6380433408 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:10:23,312 - 6380433408 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:10:23,312 - 6380433408 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

 

joaomdmoura (João Moura) · GitHub
Skip to content
Navigation Menu
Toggle navigation
 Sign in
 Product
Actions
 Automate any workflow
Packages
 Host and manage packages
Security
 Find and fix vulnerabilities
Codespaces
 Instant dev environments
Copilot
 Write better code with AI
Code review
 Manage code changes
Issues
 Plan and track work
Discussions
 Collaborate outside of code
Explore
 All features
 Documentation
 GitHub Skills
 Blog
 Solutions
For
 Enterprise
 Teams
 Startups
 Education
By Solution
 CI/CD & Automation
 DevOps
 DevSecOps
Resources
 Learning Pathways
 White papers, Ebooks, Webinars
 Customer Stories
 Partners
 Open Source
GitHub Sponsors
 Fund open source developers
The ReadME Project
 GitHub community articles
Repositories
 Topics
 Trending
 Collections
Pricing
Search or jump to...
Search code, repositories, users, issues, pull requests...
 Search
Clear
 Search syntax tips
 Provide feedback
We read every piece of feedback, and take your input very seriously.
Includ

2024-05-23 11:10:25,449 - 6363607040 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:10:25,450 - 6363607040 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:10:25,450 - 6363607040 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:10:25,450 - 6363607040 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

...

To create a set of potential interview questions and talking points tailored to Noah Williams' resume and the job requirements for the Full Stack Engineer position at AI Fund, I need to gather specific insights from Noah's resume and match them to the job description's requirements. Here is the step-by-step plan:

1. Review Noah's resume thoroughly to understand his experience, skills, and project contributions.
2. Identify key points in the job description that align with Noah's expertise.
3. Formulate interview questions that highlight Noah's relevant experiences and skills.
4. Develop talking points for Noah to showcase his qualifications effectively.

First, I'll read the entire resume content to get a complete picture of Noah's qualifications.

Action: Read a file's content
Action Input: {}
 


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technolog

2024-05-23 11:12:53,702 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:12:53,704 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:12:53,706 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:12:53,706 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

Thought: I need to delve deeper into Noah Williams' resume and compare it against the job requirements to formulate specific interview questions and talking points. I will now search the resume content for specific skill sets, projects, and experiences that align with the job requirements.

Action: Search a MDX's content
Action Input: {"search_query": "skills, projects, experiences"} 


2024-05-23 11:13:00,130 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:13:00,131 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:13:00,131 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-23 11:13:00,131 - 8021201664 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_

 

Relevant Content:
### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of Lon

- Dislplay the generated `tailored_resume.md` file.

In [19]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

**

```markdown
# Noah Williams
- **Email**: noah.williams@example.dev
- **Phone**: +44 11 111 11111

## Profile

Noah Williams is an experienced Full Stack Developer with an 18-year tenure in the technology industry, where he has excelled in building scalable software solutions for both desktop and mobile applications. He is proficient in front-end and back-end technologies, including HTML, CSS, JavaScript, Python, and Java, alongside deep expertise in frameworks such as React, Angular, and Node.js. Noah has a proven track record in developing well-functioning databases using MySQL and MongoDB, and has extensive experience in creating responsive and efficient APIs. His leadership in data science and machine learning has enabled him to integrate AI technologies seamlessly into full stack projects, making him a versatile and valuable asset to any development team.

## Skills

- **Programming Languages**: Python, JavaScript, Java, Ruby, TypeScript, Elixir
- **Front-End Technologies**: HTML, CSS, JavaScript, React, Angular, jQuery
- **Back-End Technologies**: Node.js, Ruby on Rails, Phoenix
- **Databases**: MySQL, MongoDB
- **Web Servers**: Apache
- **API Development**: RESTful APIs
- **UI/UX Design**: Responsive Design, User-Centered Design
- **Tools and Technologies**: Scalable Vector Databases, Predictive Modeling, Advanced Caching Strategies
- **Leadership**: Team Collaboration, Process Innovation, Strategic Project Management

## Project Experiences and Contributions

### GitHub Projects

1. **[crewAI](https://github.com/joaomdmoura/crewAI)**: Framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
   - **Language**: Python
   - **Stars**: 14.6k
   - **Forks**: 1.9k

2. **[machinery](https://github.com/joaomdmoura/machinery)**: Elixir State machine thin layer for structs.
   - **Language**: Elixir
   - **Stars**: 515
   - **Forks**: 53

3. **[rails-api/active_model_serializers](https://github.com/rails-api/active_model_serializers)**: ActiveModel::Serializer implementation and Rails hooks.
   - **Language**: Ruby
   - **Stars**: 5.3k
   - **Forks**: 1.4k

4. **[gioco](https://github.com/joaomdmoura/gioco)**: A gamification gem for Ruby on Rails applications.
   - **Language**: Ruby
   - **Stars**: 305
   - **Forks**: 38

5. **[sigma](https://github.com/joaomdmoura/sigma)**: A ranking algorithm implementation for Ruby on Rails applications.
   - **Language**: Ruby
   - **Stars**: 50
   - **Forks**: 5

6. **[keeper](https://github.com/joaomdmoura/keeper)**: Flexible and Simple authentication solution for Phoenix.
   - **Language**: Elixir
   - **Stars**: 30
   - **Forks**: 2

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base.
- Spearheaded the integration of cutting-edge AI technologies and scalable vector databases.
- Played a critical role in defining long-term strategic initiatives and adopting AI technologies.
- Led the development of full stack solutions, utilizing React and Node.js for front-end and back-end development respectively.
- Managed the creation and maintenance of APIs, ensuring high performance and scalability.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed engineering strategy and operations, collaborating closely with C-level executives.
- Managed diverse teams across multiple time zones, enhancing team performance and morale.
- Led full stack development projects, implementing front-end solutions with Angular and back-end services with Python.
- Developed and optimized databases using MySQL and MongoDB.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Founded InnovPet, focusing on IoT solutions for pet care.
- Oversaw product development from concept through execution.
- Built the initial version of the product leveraging MongoDB and a full stack approach using React for the front-end and Node.js for the back-end.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Formulated and executed strategic plans, enhancing inter-departmental coordination and trust.
- Managed multiple engineering teams, fostering a balance between productivity and innovation.
- Hands-on experience using Ruby on Rails and React to build out a new product.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the formation and development of a high-performance engineering team, increasing company revenue by 500%.
- Integrated data analytics into business decision-making processes, developing a predictive modeling tool.
- Developed full stack solutions, utilizing HTML, CSS, and JavaScript for front-end and Python for back-end development.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API, improving system performance and user satisfaction.
- Implemented advanced caching strategies, reducing response times and system loads.
- Utilized JavaScript and Node.js for full stack development.

### BetCraft: CTO — 2013 - 2015
- Led technology strategy post-Series A funding, guiding significant technological advancements.
- Improved platform performance and expanded market reach through strategic initiatives and partnerships.
- Developed full stack applications using both React and Angular for front-end and various back-end technologies.

## Education

- **MBA in Information Technology** - London Business School
- **Advanced Leadership Techniques Certification** - University of London
- **Data Science Specialization Certification** - Coursera (Johns Hopkins University)
- **B.Sc. in Computer Science** - University of Edinburgh
```

This tailored resume now highlights Noah Williams' qualifications and experiences that are directly relevant to the Full Stack Engineer position at AI Fund.

- Dislplay the generated `interview_materials.md` file.

In [20]:
display(Markdown("./interview_materials.md"))

Here is a document containing key questions and talking points for Noah Williams to prepare for his initial interview for the Full Stack Engineer position at AI Fund. This will help him highlight the main points of his resume and how it matches the job posting:

### Potential Interview Questions and Talking Points for Noah Williams

#### Introduction and General Questions
1. **Can you introduce yourself and provide a brief overview of your professional background?**
    - **Talking Point**: Highlight your 18-year tenure in software engineering, leadership roles, and your proficiency in various programming languages and frameworks. Mention your notable projects and contributions, especially those related to AI and machine learning.

2. **What motivated you to apply for the Full Stack Engineer position at AI Fund?**
    - **Talking Point**: Emphasize your interest in AI technologies and scalable solutions, and how AI Fund's mission aligns with your career goals and expertise in full stack development.

#### Technical Skills and Experience
3. **Can you describe your experience with front-end technologies such as HTML, CSS, and JavaScript?**
    - **Talking Point**: Discuss your proficiency in these technologies and your experience using frameworks like React and Angular. Mention specific projects where you utilized these skills.

4. **How have you utilized back-end technologies such as Node.js and Ruby on Rails in your previous roles?**
    - **Talking Point**: Provide examples from your tenure at DataKernel, InnovPet, and EliteDevs, highlighting how you developed robust back-end solutions and integrated them with front-end technologies.

5. **Can you share your experience with database management using MySQL and MongoDB?**
    - **Talking Point**: Detail your work on database optimization and management, particularly in projects like the GPS tracking collar at InnovPet and the predictive modeling tool at PrintPack.

6. **What are some of the APIs you have developed, and how did they improve system performance and user satisfaction?**
    - **Talking Point**: Highlight your work at DriveAI, where you optimized a central API, and other projects where your API development led to significant performance improvements.

#### Project Management and Leadership
7. **How have you led cross-functional teams and managed projects from conception to deployment?**
    - **Talking Point**: Discuss your leadership roles at DataKernel and PrintPack, focusing on how you managed teams, fostered collaboration, and ensured successful project execution.

8. **Can you provide an example of a challenging project you managed and how you overcame the obstacles?**
    - **Talking Point**: Share specific challenges and solutions from projects like the integration of AI technologies at DataKernel or the development of IoT solutions at InnovPet.

9. **How do you approach mentoring and developing talent within your teams?**
    - **Talking Point**: Describe your mentoring philosophy and methods, citing examples from DataKernel and EliteDevs where you successfully recruited, mentored, and retained top talent.

#### AI and Machine Learning
10. **What experience do you have with AI and machine learning, and how have you integrated these technologies into full stack projects?**
    - **Talking Point**: Discuss your significant experience in data science and AI, particularly your work on scalable AI solutions and data model development at DataKernel.

11. **Can you explain a project where you implemented AI/ML models and the impact it had on the product?**
    - **Talking Point**: Provide details of specific projects like the integration of scalable vector databases at DataKernel and how it enhanced the product's capabilities.

#### Additional Skills and Qualities
12. **How do you ensure security and data protection in your software solutions?**
    - **Talking Point**: Mention your experience in implementing security measures and data protection settings, referencing specific projects where these were critical.

13. **What strategies do you use to test and debug software to ensure responsiveness and efficiency?**
    - **Talking Point**: Share your systematic approach to testing and debugging, with examples from your work at DriveAI and other roles.

14. **How do you stay updated with the latest technologies and trends in software development?**
    - **Talking Point**: Highlight your continuous learning approach, participation in relevant courses, certifications, and involvement in tech communities.

#### Cultural Fit and Personal Attributes
15. **How do you demonstrate accountability and ownership in your work?**
    - **Talking Point**: Provide examples from your career where you took ownership of projects and delivered results, emphasizing your roles at DataKernel and InnovPet.

16. **Can you describe a situation where you exhibited grit and resilience in a challenging project?**
    - **Talking Point**: Talk about specific instances where you persevered through challenges, such as the initial funding phase at InnovPet or the technological advancements at BetCraft.

17. **How do you handle working in an early-stage or ambiguous environment?**
    - **Talking Point**: Explain your approach to navigating ambiguity and taking initiative, with examples from your startup experience at InnovPet and your leadership at EliteDevs.

#### Closing Questions
18. **Do you have any questions for us about the role or the company?**
    - **Talking Point**: Prepare thoughtful questions that show your interest in AI Fund's projects, team dynamics, and future plans.

---

This document will help Noah Williams highlight his strengths and align his experiences with the job requirements, ensuring a successful interview.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)